<a href="https://colab.research.google.com/github/maktaurus/ML-Work/blob/main/Grad_Cam_Method.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Gradient-weighted Class Activation Mapping (Grad-CAM)**
**Grad-Cam is a visual explanation algorithm.**
From a high-level, we take an image as input and create a model that is cut off at the layer for which we want to create a Grad-CAM heat-map. We attach the fully-connected layers for prediction. We then run the input through the model, grab the layer output, and prediction. Next, we find the gradient of the output of our desired model layer w.r.t. the model prediction. From there, we take sections of the gradient which contribute to the prediction, reduce, resize, and rescale so that the heat-map can be overlaid with the original image.


In [ ]:
In below code we create a function which will take image and convert that image which can be fed as per model requirement.

In [ ]:
def img_array(img_path, size):
  image = load_img(img_path, target_size=(size,size,3))
  img_arry = img_to_array(image)
  norm_img = normalize_layer(img_arry)
  # We add a dimension to transform our array into a "batch"
  # of size (1, 250, 250, 3)
  img_dims = tf.expand_dims(norm_img, axis=0)
  return img_dims

**Make Grad-Cam model**
Below function will create a grad-cam model which has following parameters:  
1.   img_array - of size desired for model
2.   model - name of the trained model
3.   layer_name - Name of the layer on which we want Grad-Cam heatmap
4.   pred_index - optional, index value of class. can be provided in situation when in a image there are more than 1 classes and we want to check if that object is present in image




In [ ]:
def Grad_Cam(model_name, layer_name, img, pred_index=None):
  # First, we create a model that maps the input image to the activations
  # of the last conv layer as well as the output predictions
  grad_model = tf.keras.Model(model_name.input, [model_name.get_layer(layer_name).output, model_name.output])

  # Then, we compute the gradient of the top predicted class for our input image
  # with respect to the activations of the last conv layer
  with tf.GradientTape() as tape:
    last_conv_layer_output, pred = grad_model(img)
    if pred_index is None:
      pred_index = np.argmax(pred)
    class_channel = pred[:,pred_index]
    dog_breed = class_name[pred_index]
    print(dog_breed)

  # This is the gradient of the output neuron (top predicted or chosen)
  # with regard to the output feature map of the last conv layer
  grads = tape.gradient(class_channel, last_conv_layer_output)

  # This is a vector where each entry is the mean intensity of the gradient
  # over a specific feature map channel
  pooled_grads = tf.reduce_mean(grads, axis=(0,1,2))

  # We multiply each channel in the feature map array
  # by "how important this channel is" with regard to the top predicted class
  # then sum all the channels to obtain the heatmap class activation
  heatmap = last_conv_layer_output[0] @ pooled_grads[...,tf.newaxis]
  heatmap = tf.squeeze(heatmap)

  # For visualization purpose, we will also normalize the heatmap between 0 & 1
  heatmap = tf.nn.relu(heatmap).numpy()
  return heatmap

**Superimposed Image Visualization**
Below function will take the output heatmap from previous Grad-Cam model and imposes on orginal input image. This will tells us where our model is looking to make prediction.

In [ ]:
import matplotlib as mpl
def superimpose_image(heatmap, img_path, alpha=0.5):
  img = tf.keras.utils.load_img(img_path)
  img = tf.keras.utils.img_to_array(img)

  heatmap = np.uint8(255*heatmap)
  heatmap = np.uint8(255*heatmap)
  jet = mpl.colormaps["jet"]
  jet_colors = jet(np.arange(256))[:,:3]
  jet_heatmap = jet_colors[heatmap]

  jet_heatmap = tf.keras.utils.array_to_img(jet_heatmap)
  jet_heatmap = tf.image.resize(jet_heatmap, (img.shape[0],img.shape[1]))
  jet_heatmap = tf.keras.utils.img_to_array(jet_heatmap)

  superimposed_img = jet_heatmap * alpha + img
  superimposed_img = tf.keras.utils.array_to_img(superimposed_img)

  return plt.imshow(superimposed_img)

Below is the example, where we have loaded the Xception model and predicted the calss and its heatmap.
Note: When using pretained model deactivate the last activation layer.

In [ ]:
from tensorflow.keras.applications.xception import Xception, decode_predictions, preprocess_input

pretrained_model = Xception(weights="imagenet")

pretrained_model.layers[-1].activation = None


In [ ]:
img = preprocess_input(img_array("/content/testing/q19.jpg",299))

pred = pretrained_model.predict(img)

print(decode_predictions(pred, top=2))
print(img.shape)

In [ ]:
img = img_array("/content/testing/q19.jpg", 299)
heatmap = Grad_Cam(pretrained_model, "block14_sepconv2_act", img,208)
superimpose_image(heatmap,"/content/testing/q19.jpg",3)

Below Example, where you are using a pretained model trained on your custom dataset.
Note: here you dont have to deactivate the last activation layer.
Provide image size as desired by model

In [ ]:
img = img_array("/content/testing/q19.jpg", 299)
heatmap = Grad_Cam(pretrained_model, "block14_sepconv2_act", img)
superimpose_image(heatmap,"/content/testing/q19.jpg",1)